In [2]:
## create ground truth label from batch results
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join


In [5]:
## Load the CSV files and save in 1 dataframe
csv_files = [f for f in listdir("batch_results_oracle") if isfile(join("batch_results_oracle", f))]

# all_results = pd.DataFrame(columns=["anchor_id", "anchor_label", image_id", "melanoma"])
all_results = pd.DataFrame()

for i in csv_files:
    temp_df = pd.read_csv("batch_results_oracle/"+i)
    all_results = all_results.append(temp_df, ignore_index=True)
    

# print(all_results.columns)

## Drop all rows we don't need
df_prep0 = all_results.drop(columns=['HITId', 'HITTypeId', 'Title', 'Description', 'Keywords', 'Reward',
       'CreationTime', 'MaxAssignments', 'RequesterAnnotation',
       'AssignmentDurationInSeconds', 'AutoApprovalDelayInSeconds',
       'Expiration', 'NumberOfSimilarHITs', 'LifetimeInSeconds',
       'AssignmentId', 'WorkerId', 'AssignmentStatus', 'AcceptTime',
       'SubmitTime', 'AutoApprovalTime', 'ApprovalTime', 'RejectionTime',
       'RequesterFeedback', 'WorkTimeInSeconds', 'LifetimeApprovalRate',
       'Last30DaysApprovalRate', 'Last7DaysApprovalRate', 'Input.id',
       'Input.item1_url', 'Input.item2_label', 'Input.item2_url', 'Input.item3_label', 'Input.item3_url','Approve', 'Reject'])

# Rename columns
df_prep0 = df_prep0.rename(columns={"Input.item1_title":"anchor_id", "Input.item1_label" : "anchor_label", "Input.item2_title":"item2", "Input.item3_title":"item3", "Answer.equal.label":"answer"  })
print(df_prep0)

## Create the ground truth file
# Step 1: give image_id to picture that has been answered

df_prep1 = df_prep0
## Find image ID based on the answer is A or N
df_prep1[["image_id"]] = df_prep1[["item2"]].where(df_prep1[["answer"]].values=="A")
## Because it is binary, we now all NaN is B (Image 3)
df_prep1.image_id.fillna(df_prep1.item3,inplace=True)
#print(df_prep1)

# Step 2: define the label equal to the anchor
df_prep2 = pd.DataFrame()
df_prep2[["image_id", "melanoma"]] = df_prep1[["image_id", "anchor_label"]]
df_prep2 = df_prep2.sort_values("image_id")
print(df_prep2)

# Step 3: Take average of all answers 
df_prep3 = df_prep2
df_prep3[["melanoma"]] = df_prep3["melanoma"].astype(float)
df_prep3 = df_prep3.groupby("image_id").mean()
print(df_prep3)

# Step 4: Round to nearest label {0,1}
df = df_prep3
df = df.round({'melanoma': 0})
print(df)


        anchor_id  anchor_label         item2         item3 answer
0    ISIC_0000484           1.0  ISIC_7228524  ISIC_9556334      B
1    ISIC_0010227           0.0  ISIC_6355353  ISIC_0953317      B
2    ISIC_0013845           1.0  ISIC_2609984  ISIC_6695831      A
3    ISIC_0000081           0.0  ISIC_3065032  ISIC_7748311      B
4    ISIC_0009898           1.0  ISIC_4710645  ISIC_3628451      B
..            ...           ...           ...           ...    ...
496  ISIC_0009993           1.0  ISIC_7186503  ISIC_4653503      B
497  ISIC_0013578           1.0  ISIC_1648826  ISIC_7800750      B
498  ISIC_0012735           0.0  ISIC_5829938  ISIC_5021861      B
499  ISIC_0000463           1.0  ISIC_6671773  ISIC_6255113      A
500           NaN           NaN           NaN           NaN    NaN

[501 rows x 5 columns]
         image_id  melanoma
306  ISIC_0149568       1.0
469  ISIC_0188432       1.0
367  ISIC_0234096       1.0
93   ISIC_0236510       0.0
477  ISIC_0272509       0.0
..  

In [6]:
## Save ground truth file to csv
df.to_csv("ORACLE_2021_GroundTruth.csv")